In [1]:
import pandas as pd
import re
import os

In [2]:
df = pd.read_csv("./instrumentation-candidates-pdfbox.csv")
df.shape

(137, 15)

In [3]:
df.head()

,visibility,parent-FQN,method-name,param-list,return-type,local-variables,conditionals,multiple-statements,loops,parameters,returns,switches,ifs,static,returns-primitives
0,public,org.apache.fontbox.ttf.VerticalMetricsTable,getAdvanceHeight,int,int,False,False,False,False,True,True,False,True,False,True
1,public,org.apache.fontbox.util.BoundingBox,contains,"float, float",boolean,False,False,False,False,True,True,False,False,False,True
2,private,org.apache.pdfbox.pdmodel.encryption.AccessPer...,isPermissionBitOn,int,boolean,False,False,False,False,True,True,False,False,False,True
3,public,org.apache.pdfbox.util.SmallMap,size,NaN,int,False,True,False,False,False,True,False,False,False,True
4,private,org.apache.pdfbox.tools.PDFText2HTML$FontState,openTag,java.lang.String,java.lang.String,False,False,False,False,True,True,False,False,False,False


In [4]:
# Execute workloads
for index, row in df.iterrows():
    method_fqn = str(row['parent-FQN']) + "." + str(row['method-name'])
    workloads = ["Encrypt", "Decrypt", "ExtractText", "ExtractImages", "PDFToImage", "TextToPDF", "PDFSplit", "PDFMerger", "WriteDecodedDoc", "OverlayPDF"]
    for i in range(len(workloads)):
        current_workload = workloads[i]
        dir_name = "/home/user/dev/async-profiler/pdfbox-profiling/" + current_workload
        
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)
        
        if current_workload == "Encrypt":
            pdfbox_cmd = "Encrypt -O 123 -U 123 /home/user/panktigen-workload/new-workload.pdf /home/user/panktigen-workload/new-workload-locked.pdf"
            
        elif current_workload == "Decrypt":
            pdfbox_cmd = "Decrypt -password 123 /home/user/panktigen-workload/new-workload-locked.pdf /home/user/panktigen-workload/new-workload-unlocked.pdf"
        
        elif current_workload == "ExtractText":
            pdfbox_cmd = "ExtractText -password 123 -sort /home/user/panktigen-workload/new-workload-locked.pdf /home/user/panktigen-workload/new-workload-from-pdf.txt"
        
        elif current_workload == "ExtractImages":
            pdfbox_cmd = "ExtractImages -password 123 -directJPEG /home/user/panktigen-workload/new-workload-locked.pdf"
        
        elif current_workload == "PDFToImage":
            pdfbox_cmd = "PDFToImage -password 123 -imageType png -outputPrefix workloadAsImage -time /home/user/panktigen-workload/new-workload-locked.pdf"
        
        elif current_workload == "TextToPDF":
            pdfbox_cmd = "TextToPDF -standardFont Helvetica-BoldOblique -fontSize 18 /home/user/panktigen-workload/new-workload-from-txt.pdf /home/user/panktigen-workload/new-workload-from-pdf.txt"
        
        elif current_workload == "PDFSplit":
            pdfbox_cmd = "PDFSplit -password 123 -split 1 /home/user/panktigen-workload/new-workload-locked.pdf"
        
        elif current_workload == "PDFMerger":
            pdfbox_cmd = "PDFMerger /home/user/panktigen-workload/new-workload-locked-*.pdf /home/user/panktigen-workload/new-workload-merged.pdf"
        
        elif current_workload == "WriteDecodedDoc":
            pdfbox_cmd = "WriteDecodedDoc -password 123 /home/user/panktigen-workload/new-workload-locked.pdf /home/user/panktigen-workload/new-workload-decoded.pdf"
        
        elif current_workload == "OverlayPDF":
            pdfbox_cmd = "OverlayPDF /home/user/panktigen-workload/new-workload.pdf /home/user/panktigen-workload/overlay.pdf -position foreground /home/user/panktigen-workload/new-workload-overlaid.pdf"
            
        output_file = dir_name + "/" + method_fqn + "-%t.html"
        cmd = "java -XX:+UnlockDiagnosticVMOptions -XX:+DebugNonSafepoints -agentpath:/home/user/dev/async-profiler/build/libasyncProfiler.so=start,tree,file=" + output_file + ",event=" + method_fqn + ",interval=1,alluser,dot,threads,framebuf=5000000 -jar /home/user/dev/pdfbox/app/target/pdfbox-app-2.0.20.jar " + pdfbox_cmd
        output = os.system(cmd)
        if (output != 0):
            print("Profiling error for " + method_fqn + " for workload " + current_workload)